In [3]:
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, BertForRetrieval,
    BertForSequenceClassification)
import torch
import numpy as np

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
ckpt_dir = "/home/transformers-public/output_dirs/classification_test/checkpoint-4500"
# ckpt_dir = "../output_dirs/classification_test/checkpoint-4500"

In [5]:
config = AutoConfig.from_pretrained(ckpt_dir)
tokenizer = BertTokenizer.from_pretrained(
    ckpt_dir,
    do_lower_case=True,
)
model = BertForSequenceClassification.from_pretrained(
    ckpt_dir,
    from_tf=False,
    config=config,
)

In [20]:
use_gpu = False
if use_gpu:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cpu


In [10]:
model = model.to(device)

In [11]:
max_seq_length=128
test_input = tokenizer.batch_encode_plus(
    [("test input", None)],
    max_length=max_seq_length,
    pad_to_max_length=True,
)

In [12]:
model_inp = dict(test_input)
model_inp['labels'] = [0]
# print(model_inp)
model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
model_out = model(**model_inp)

In [13]:
len(model_out)

3

In [14]:
print([x.shape for x in model_out])  # loss, logits, pooled_output (i.e. encoding)

[torch.Size([]), torch.Size([1, 30]), torch.Size([1, 768])]


In [15]:
predicted_logits = model_out[1].detach().cpu().numpy()
predicted_class = np.argmax(predicted_logits)

In [16]:
print(predicted_logits)

[[-1.3015643   2.019937    2.3875947  -4.169917   -1.4164282  -4.2287807
  -1.8702357   0.35857603 -2.7242658  -2.6567934  -1.8068227  -2.017031
  -1.5032881  -2.942142    0.22073309 -0.82663375 -2.202315    0.8912497
   2.5810022   0.01150044  0.6968271  -0.89142245  0.89638424 -0.4870008
   2.7677815  -2.8281648  -2.5763004  -0.8343036   0.36374706 -0.04874416]]


In [17]:
print(predicted_class)

24
